# **Herramienta de Agricultura Urbana**
# Ésta herramienta busca facilitar el comienzo de un proyecto de Agricultura Urbana. Considera un espacio que tengas disponible en tu entorno y contesta las preguntas. Esta es una versión de prueba, con el fin de observar su uso.


In [0]:
#@markdown Código Postal (5 dígitos)
codigo_postal =   66220 #@param {type:"integer"}
#@markdown Tipo de Comunidad
tipo_comunidad = "Familiar" #@param ["Laboral", "Familiar", "Vecinal", "Educativa"]
#@markdown Tamaño de Comunidad (personas)
tamaño_comunidad = "6 - 20" #@param ["1 - 5", "6 - 20", "20 +"]
#@markdown Tipo de Consumo
tipo_consumo = "Personal" #@param ["Personal", "Fines Lucrativos (venta)", "Educativo"]
#@markdown ¿Área Disponible para Cultivar?
area_cultivo_m2 =  40#@param {type:"integer"}
#@markdown ¿Luz Solar Directa?
luz_solar = True #@param ["True", "False"] {type:"raw"}
#@markdown ¿Acceso a Agua para Riego?
agua_riego = "Llave" #@param ["Llave", "Pipa o Cisterna", "No"]
acceso_electricidad = True #@param ["True", "False"] {type:"raw"}
#@markdown ¿Fuente de materia orgánica compostable cercana?
acceso_composta = True #@param ["True", "False"] {type:"raw"}


# Información

In [0]:
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
!pip install pgeocode
import pgeocode
import json 

def cargar_sistema():

  # FECHA HOY
  base = d = datetime.today() - timedelta(days=1)
  #FECHA HOY STRING
  fecha = str(base).split()[0]
  #AÑO END
  año_end = fecha[:-6]
  #MES
  mes_inicio = fecha[5:-3]
  #DÍA
  dia_inicio = fecha[8:]
  #AÑO START
  año_start = int(año_end)-4


  start_date = f"{año_start}{mes_inicio}{dia_inicio}"
  end_date = f"{año_end}{mes_inicio}{dia_inicio}"

  ## RECUPERAR LATITUD Y LONGITUD MEDIANTE UN API DE GEOCÓDIGO (USANDO CP)
  latitud = pgeocode.Nominatim('MX').query_postal_code(codigo_postal).latitude
  longitud = pgeocode.Nominatim('MX').query_postal_code(codigo_postal).longitude

  link = f"https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=T2M_RANGE,T2M_MAX,T2M_MIN,T2M,ALLSKY_SFC_SW_DWN&startDate={start_date}&endDate={end_date}&userCommunity=AG&tempAverage=DAILY&outputList=CSV&lat={latitud}&lon={longitud}"

  r = requests.get(link)
  j = r.json()
  df = pd.DataFrame(j, columns= ['outputs']) ## GUARDAR DATOS EN UN DATAFRAME
  link
  csv = df.iloc[0,0]
  datos = pd.read_csv(csv,skiprows=14)

  nuevo = float()
  pasado = float()
  
  if datos['ALLSKY_SFC_SW_DWN'].mean() < 10: nuevo = "Inverdadero - Alta"
  else: nuevo = "Orgánico Exterior"
  pasado = nuevo
    
  if datos['T2M_RANGE'].mean() >= 10: nuevo = "Inverdadero - Media"
  else: nuevo = pasado
  pasado = nuevo
    
  if datos['T2M_MAX'].mean() >= 29: nuevo = "Inverdadero - Alta"
  else: nuevo = pasado
  pasado = nuevo
    
  if datos['T2M_MIN'].mean() <= 10: nuevo = "Inverdadero - Alta"
  else: nuevo = pasado
  pasado = nuevo
  
  if datos['T2M'].mean() >= 25: nuevo = "Inverdadero - Alta"
  else: nuevo = pasado
  pasado = nuevo
 
  if ((datos['T2M'].mean()) + ((datos['T2M_RANGE'].mean())/2)) >= 28: nuevo = "Inverdadero - Alta"
  else: nuevo = pasado
  pasado = nuevo
 
  if ((datos['T2M'].mean()) - ((datos['T2M_RANGE'].mean())/2)) <= 12: nuevo = "Inverdadero - Alta"
  else: nuevo = pasado
  pasado = nuevo
 
  if (datos['T2M_MAX'].max()) >= 30: nuevo = pasado
  else: nuevo = "Inverdadero - Baja"
  pasado = nuevo
 
  if (datos['T2M_MIN'].min()) <= 9: nuevo = pasado
  else: nuevo = "Inverdadero - Baja"
  pasado = nuevo
 
  if tamaño_comunidad == "20 +": nuevo = "Inverdadero - Alta"
  else: nuevo = pasado
  pasado = nuevo
 
  if area_cultivo_m2 <= 50: nuevo = "Inverdadero - Baja"
  else: nuevo = pasado
  pasado = nuevo
 
  if luz_solar == True: nuevo = pasado
  else: nuevo = "Inverdadero - Alta"
  pasado = nuevo
 
  if agua_riego == "Llave": nuevo = pasado
  else: nuevo = "Hidropónico"
 
  if nuevo == pasado: nuevo = "Por Goteo"
  else: nuevo = "Hidropónico"
  pasado = nuevo
  nuevo1 = nuevo
 
  if agua_riego == "No": nuevo = "Ciclo Cerrado"
  else: nuevo = pasado
  pasado = nuevo
  nuevo2 = nuevo
 
  if acceso_electricidad == False: nuevo = "Inverdadero - Baja"
  else: nuevo = pasado
  pasado = nuevo
 
  if nuevo == "Inverdadero - Baja": nuevo = "Por Goteo"
  else: nuevo = nuevo1
  pasado = nuevo
  
  if nuevo == "Por Goteo": nuevo = "Por Goteo"
  else: nuevo = nuevo2
  pasado = nuevo
 
  if tipo_consumo == "Fines Lucrativos (venta)" and (tamaño_comunidad == "20 +"): nuevo = "Inverdadero - Alta"
  else: nuevo = pasado
  pasado = nuevo
 
  if nuevo1 == "Hidropónico": nuevo = "Hidropónico"
  else: nuevo = "Hidropónico"
  pasado = nuevo
  nuevo3 = nuevo

  if nuevo2 == "Ciclo Cerrado": nuevo = "Ciclo Cerrado"
  else: nuevo = "Ciclo Cerrado"
  pasado = nuevo
  nuevo4 = nuevo

  if acceso_composta == False: nuevo = "Hidropónico"
  else: nuevo = pasado
  pasado = nuevo
  nuevo5 = nuevo

  if nuevo == "Hidropónico": nuevo = "Hidropónico"
  else: nuevo = nuevo3
  pasado = nuevo
  nuevo6 = nuevo

  if nuevo4 == "Ciclo Cerrado": nuevo = "Ciclo Cerrado"
  else: nuevo = "Ciclo Cerrado"
  pasado = nuevo
  nuevo7 = nuevo

  if tipo_comunidad == "Vecinal" and tamaño_comunidad <= 20: nuevo = "Inverdadero - Media"
  elif tipo_comunidad == "Laboral": nuevo = "Inverdadero - Alta"
  else: nuevo = nuevo5
  pasado = nuevo
  nuevo8 = nuevo

  if nuevo6 == "Hidropónico": nuevo = "Hidropónico"
  else: nuevo = "Hidropónico"
  pasado = nuevo

  if nuevo7 == "Ciclo Cerrado": nuevo = "Ciclo Cerrado"
  else: nuevo = pasado

  if nuevo == "Ciclo Cerrado": nuevo = "Hidropónico"
  else: nuevo = nuevo8
  pasado = nuevo
  nuevo9 = nuevo

  if nuevo == "Inverdadero - Baja": crecimiento = 12 ## kg/m2 - bajo
  elif nuevo == "Inverdadero - Alta": crecimiento = 60 ## kg/m2 - bajo
  else: crecimiento = 22.5 ## kg/m2 - bajo
  
  if nuevo9 == "Inverdadero - Baja": consumo_agua = 40 ## consumo lt/kg
  elif nuevo9 == "Inverdadero - Media": consumo_agua = 30 ## consumo lt/kg
  elif nuevo9 == "Inverdadero - Alta": consumo_agua = 22 ## consumo lt/kg
  elif nuevo9 == "Ciclo Cerrado": consumo_agua = 4 ## consumo lt/kg ciclo cerrado no agua
  else: consumo_agua = 15 ## consumo lt/kg

  if nuevo == "Inverdadero - Baja": costo_estimado_proyecto = 159.33*area_cultivo_m2 ## costo/m2 - bajo
  elif nuevo == "Inverdadero - Media": costo_estimado_proyecto = 464.57*area_cultivo_m2 ## costo/m2 - medio
  elif nuevo == "Hidropónico": costo_estimado_proyecto = 464.57*area_cultivo_m2 ## costo/m2 - medio
  elif nuevo == "Ciclo Cerrado": costo_estimado_proyecto = 1128.66*area_cultivo_m2 ## costo/m2 - alto
  else: costo_estimado_proyecto = 1128.66*area_cultivo_m2

  recomendacion = str(nuevo)
  produccion_kg = crecimiento*area_cultivo_m2
  valor_estimado_producto = produccion_kg*24.9 # PRECIO "TOMATE"
  consumo_agua_lt = consumo_agua*produccion_kg